In [1]:
import pandas as pd
import numpy as np

In [2]:
#read in all 3 csvs - its time to combine them into one!
nadac = pd.read_csv('./data/nadac_no_repeats.csv')
orange = pd.read_csv('./data/drug_names_df.csv')
matches = pd.read_csv('./data/names_df_no_overwrites_long.csv')

In [3]:
matches.head()

,Unnamed: 0,names
0,ENSTILAR 0.005%-0.064% FOAM,ENSTILAR 0.064%;0.005% TOPICAL
1,DOXYCYCLINE HYCLATE 100MG CAPSULE ORAL,DOXYCYCLINE HYCLATE EQ 150MG BASE ORAL
2,DOXYCYCLINE HYCLATE 50MG CAPSULE ORAL,DOXYCYCLINE HYCLATE EQ 80MG BASE ORAL
3,AGGRENOX 25MG-200MG CAPSULE,AGGRENOX 25MG;200MG ORAL
4,CARBAMAZEPINE ER 100MG TABLET,CARBAMAZEPINE 100MG ORAL


In [4]:
matches.rename(columns={'Unnamed: 0': 'nadac_name', 'names':'orange_book_name'}, inplace=True)
matches.head()

,nadac_name,orange_book_name
0,ENSTILAR 0.005%-0.064% FOAM,ENSTILAR 0.064%;0.005% TOPICAL
1,DOXYCYCLINE HYCLATE 100MG CAPSULE ORAL,DOXYCYCLINE HYCLATE EQ 150MG BASE ORAL
2,DOXYCYCLINE HYCLATE 50MG CAPSULE ORAL,DOXYCYCLINE HYCLATE EQ 80MG BASE ORAL
3,AGGRENOX 25MG-200MG CAPSULE,AGGRENOX 25MG;200MG ORAL
4,CARBAMAZEPINE ER 100MG TABLET,CARBAMAZEPINE 100MG ORAL


In [5]:
#Make this back into a dictionary to be able to use .map
name_matches = matches.set_index('nadac_name')['orange_book_name'].to_dict()

In [6]:
#Make a column with the matching names
nadac['orange_names'] = nadac['NDC Description'].map(name_matches)
nadac.head()

,Unnamed: 0,NDC Description,NDC,NADAC_Per_Unit,Effective_Date,Pricing_Unit,Pharmacy_Type_Indicator,OTC,Explanation_Code,Classification_for_Rate_Setting,Corresponding_Generic_Drug_NADAC_Per_Unit,Corresponding_Generic_Drug_Effective_Date,As of Date,orange_names
0,152,OLANZAPINE ODT 20MG TABLET,65862065903,4.12341,12/23/2015,EA,C/I,N,1,G,NaN,NaN,12/30/2015,NaN
1,213,"NYSTATIN 100,000 UNIT/GM POWD",904645602,0.60225,12/23/2015,GM,C/I,N,"1, 5",G,NaN,NaN,12/30/2015,NaN
2,298,OLANZAPINE 10MG TABLET,904637661,0.22020,12/23/2015,EA,C/I,N,1,G,NaN,NaN,12/30/2015,OLANZAPINE 10MG ORAL
3,575,CELECOXIB 50MG CAPSULE,13811065860,0.51850,11/11/2015,EA,C/I,N,4,G,NaN,NaN,12/30/2015,NaN
4,1168,LISINOPRIL 20MG TABLET,51079098319,0.02633,12/17/2014,EA,C/I,N,1,G,NaN,NaN,12/31/2014,LISINOPRIL 20MG ORAL


In [7]:
nadac.drop(columns=['Unnamed: 0'], inplace=True)

In [8]:
nadac['orange_names'].isnull().sum()

35530

In [9]:
#remove all data that does not have a matching drug name
nadac.dropna(subset=['orange_names'], inplace = True)

In [10]:
#make both indexes the matching columns to merge
nadac.set_index('orange_names', inplace=True)
orange.set_index('ndc_description_col', inplace = True)

In [11]:
nadac.head()

,NDC Description,NDC,NADAC_Per_Unit,Effective_Date,Pricing_Unit,Pharmacy_Type_Indicator,OTC,Explanation_Code,Classification_for_Rate_Setting,Corresponding_Generic_Drug_NADAC_Per_Unit,Corresponding_Generic_Drug_Effective_Date,As of Date
orange_names,,,,,,,,,,,,
OLANZAPINE 10MG ORAL,OLANZAPINE 10MG TABLET,904637661,0.22020,12/23/2015,EA,C/I,N,1,G,NaN,NaN,12/30/2015
LISINOPRIL 20MG ORAL,LISINOPRIL 20MG TABLET,51079098319,0.02633,12/17/2014,EA,C/I,N,1,G,NaN,NaN,12/31/2014
ALLOPURINOL 100MG ORAL,ALLOPURINOL 100MG TABLET,62584098811,0.17160,12/17/2014,EA,C/I,N,1,G,NaN,NaN,12/31/2014
GABAPENTIN 600MG ORAL,GABAPENTIN 600MG TABLET,76282040501,0.36021,12/24/2014,EA,C/I,N,3,G,NaN,NaN,12/31/2014
GABAPENTIN 800MG ORAL,GABAPENTIN 800MG TABLET,378542501,0.21958,12/17/2014,EA,C/I,N,1,G,NaN,NaN,12/31/2014


In [12]:
#Merge!
all_drugs = pd.merge(nadac, orange, left_on= nadac.index, right_on = orange.index)

In [13]:
all_drugs.head()

,key_0,NDC Description,NDC,NADAC_Per_Unit,Effective_Date,Pricing_Unit,Pharmacy_Type_Indicator,OTC,Explanation_Code,Classification_for_Rate_Setting,...,Appl_No,Product_No,TE_Code,Approval_Date,RLD,RS,Type,Applicant_Full_Name,dosage_form,route
0,OLANZAPINE 10MG ORAL,OLANZAPINE 10MG TABLET,904637661,0.2202,12/23/2015,EA,C/I,N,1,G,...,204320,2,NaN,"May 30, 2017",No,No,DISCN,AJANTA PHARMA LTD,"TABLET, ORALLY DISINTEGRATING",ORAL
1,OLANZAPINE 10MG ORAL,OLANZAPINE 10MG TABLET,904637661,0.2202,12/23/2015,EA,C/I,N,1,G,...,91265,2,AB,"Oct 24, 2011",No,No,RX,APOTEX INC,"TABLET, ORALLY DISINTEGRATING",ORAL
2,OLANZAPINE 10MG ORAL,OLANZAPINE 10MG TABLET,904637661,0.2202,12/23/2015,EA,C/I,N,1,G,...,203708,2,AB,"May 15, 2014",No,No,RX,AUROBINDO PHARMA LTD,"TABLET, ORALLY DISINTEGRATING",ORAL
3,OLANZAPINE 10MG ORAL,OLANZAPINE 10MG TABLET,904637661,0.2202,12/23/2015,EA,C/I,N,1,G,...,77243,2,AB,"Jan 30, 2012",No,No,RX,BARR LABORATORIES INC,"TABLET, ORALLY DISINTEGRATING",ORAL
4,OLANZAPINE 10MG ORAL,OLANZAPINE 10MG TABLET,904637661,0.2202,12/23/2015,EA,C/I,N,1,G,...,76534,2,AB,"Oct 24, 2011",No,No,RX,DR REDDYS LABORATORIES LTD,"TABLET, ORALLY DISINTEGRATING",ORAL


In [14]:
all_drugs.set_index('key_0', inplace=True)
all_drugs.head()

,NDC Description,NDC,NADAC_Per_Unit,Effective_Date,Pricing_Unit,Pharmacy_Type_Indicator,OTC,Explanation_Code,Classification_for_Rate_Setting,Corresponding_Generic_Drug_NADAC_Per_Unit,...,Appl_No,Product_No,TE_Code,Approval_Date,RLD,RS,Type,Applicant_Full_Name,dosage_form,route
key_0,,,,,,,,,,,,,,,,,,,,,
OLANZAPINE 10MG ORAL,OLANZAPINE 10MG TABLET,904637661,0.2202,12/23/2015,EA,C/I,N,1,G,NaN,...,204320,2,NaN,"May 30, 2017",No,No,DISCN,AJANTA PHARMA LTD,"TABLET, ORALLY DISINTEGRATING",ORAL
OLANZAPINE 10MG ORAL,OLANZAPINE 10MG TABLET,904637661,0.2202,12/23/2015,EA,C/I,N,1,G,NaN,...,91265,2,AB,"Oct 24, 2011",No,No,RX,APOTEX INC,"TABLET, ORALLY DISINTEGRATING",ORAL
OLANZAPINE 10MG ORAL,OLANZAPINE 10MG TABLET,904637661,0.2202,12/23/2015,EA,C/I,N,1,G,NaN,...,203708,2,AB,"May 15, 2014",No,No,RX,AUROBINDO PHARMA LTD,"TABLET, ORALLY DISINTEGRATING",ORAL
OLANZAPINE 10MG ORAL,OLANZAPINE 10MG TABLET,904637661,0.2202,12/23/2015,EA,C/I,N,1,G,NaN,...,77243,2,AB,"Jan 30, 2012",No,No,RX,BARR LABORATORIES INC,"TABLET, ORALLY DISINTEGRATING",ORAL
OLANZAPINE 10MG ORAL,OLANZAPINE 10MG TABLET,904637661,0.2202,12/23/2015,EA,C/I,N,1,G,NaN,...,76534,2,AB,"Oct 24, 2011",No,No,RX,DR REDDYS LABORATORIES LTD,"TABLET, ORALLY DISINTEGRATING",ORAL


In [15]:
all_drugs.columns

Index(['NDC Description', 'NDC', 'NADAC_Per_Unit', 'Effective_Date',
       'Pricing_Unit', 'Pharmacy_Type_Indicator', 'OTC', 'Explanation_Code',
       'Classification_for_Rate_Setting',
       'Corresponding_Generic_Drug_NADAC_Per_Unit',
       'Corresponding_Generic_Drug_Effective_Date', 'As of Date', 'Unnamed: 0',
       'Ingredient', 'DF;Route', 'Trade_Name', 'Applicant', 'Strength',
       'Appl_Type', 'Appl_No', 'Product_No', 'TE_Code', 'Approval_Date', 'RLD',
       'RS', 'Type', 'Applicant_Full_Name', 'dosage_form', 'route'],
      dtype='object')

In [16]:
all_drugs['NDC Description'].value_counts()

IBUPROFEN 200MG TABLET                  4838
LEVETIRACETAM 500MG TABLET              2726
LEVETIRACETAM 750MG TABLET              2303
HYDROCHLOROTHIAZIDE 50MG TABLET ORAL    1932
LOSARTAN POTASSIUM 50MG TABLET ORAL     1911
                                        ... 
FYCOMPA 0.5MG/ML ORAL SUSP                 1
ZIRGAN 0.15% OPHTHALMIC GEL                1
PROGLYCEM 50MG/ML ORAL SUSP                1
TEGRETOL XR 400MG TABLET                   1
NEURONTIN 600MG TABLET                     1
Name: NDC Description, Length: 598, dtype: int64

In [17]:
all_drugs.dtypes

NDC Description                               object
NDC                                            int64
NADAC_Per_Unit                               float64
Effective_Date                                object
Pricing_Unit                                  object
Pharmacy_Type_Indicator                       object
OTC                                           object
Explanation_Code                              object
Classification_for_Rate_Setting               object
Corresponding_Generic_Drug_NADAC_Per_Unit    float64
Corresponding_Generic_Drug_Effective_Date     object
As of Date                                    object
Unnamed: 0                                     int64
Ingredient                                    object
DF;Route                                      object
Trade_Name                                    object
Applicant                                     object
Strength                                      object
Appl_Type                                     

In [18]:
all_drugs.to_csv('./data/all_drugs.csv')

In [19]:
brand_names = all_drugs[(all_drugs['Classification_for_Rate_Setting'] == 'B')|(all_drugs['Classification_for_Rate_Setting'] == 'B_ANDA')]

In [20]:
brand_names.shape

(226, 29)

In [21]:
brand_names.to_csv('./data/top_brand_names.csv')

In [ ]:
g